### Sequence 나누는 함수
- backtesting 용

In [1]:
# 필요 라이브러리 import

# Pytorch
import torch
import torch.optim as optim
import torch.nn as nn

# Dataset 관련
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import sequence as sq # 사용자 정의 함수 불러오기

# 성능 평가 관련
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
from collections import Counter

# Visualization 관련
import matplotlib.pyplot as plt
import seaborn as sns

# 하이퍼파라미터 튜닝
import optuna
from optuna.pruners import MedianPruner
from sklearn.model_selection import TimeSeriesSplit

# 운영체제 관련
import platform

'''
딥러닝 학습을 진행할 때, 가중치를 임의의 값으로 초기화하여 학습을 수행하는 데, 
실험을 동일하게 진행하기 위해서는 난수를 동일하게 생성해야 한다.
Pytorch에서 random seed를 고정하기 위해 manual_seed를 사용한다.
'''

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# 운영체제별 device 설정
os_name = platform.system()
if os_name == 'Windows':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"이 PC는 윈도우 운영 체제입니다. {device} is available")
elif os_name == 'Darwin':
    device = torch.device("mps" if torch.backends.mps.is_available else "cpu")
    print(f"이 PC는 맥(OS X) 운영 체제입니다. {device} is available")
else:
    print(f"이 PC는 다른 운영 체제입니다: {os_name}")


이 PC는 맥(OS X) 운영 체제입니다. mps is available


/opt/homebrew/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 데이터 불러오기
file_path = '../../data/' # 경로 설정
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
df = df.iloc[:50000]
df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) # 종속변수 이진분류화
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬
df = df.reset_index() # backtesting을 위한 index 값 생성

# sequence length를 기준으로 sequence 데이터 생성
seq_len = 20 # 20, 40, 80, 160, 320
X, y = sq.create_sequence(df, seq_len=seq_len) # 사용자 정의 함수
# Tensor화
X_torch = torch.FloatTensor(X).to(device)
y_torch = torch.FloatTensor(y).to(device)
print('Full Data Size:', X_torch.size(), y_torch.size())

# split (80% / 20%)
train_split = int((X_torch.size(0)) * 0.8)

X_train_seq = X_torch[:train_split]
X_test_seq = X_torch[train_split:]
y_train_seq = y_torch[:train_split]
y_test_seq = y_torch[train_split:]

print('Train Size:', X_train_seq.size(), y_train_seq.size())
print('Test Size:', X_test_seq.size(), y_test_seq.size())


Full Data Size: torch.Size([21387, 20, 78]) torch.Size([21387, 1])
Train Size: torch.Size([17109, 20, 78]) torch.Size([17109, 1])
Test Size: torch.Size([4278, 20, 78]) torch.Size([4278, 1])


In [3]:
"""
Note: bitcoin_prepro_v2.ipynb 파일을 통해 생성된 df에 적용할 것
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_sequence(df, seq_len):
    # 1. 변수 선택
    # 종속변수 리스트
    #target_var_lst = ['returns', 'returns_next10m', 'realized_vol_next10m']
    #target_var = 'returns_next10m' # 종속변수

    # 시퀀스 생성 전 필요없는 컬럼 삭제
    df.drop(columns=['window_start', 'window_end','num_rows', 'time_id'], inplace=True) # 시간 관련 변수

    # target을 제외한 나머지 종속변수 삭제
    #cols_to_drop = [var for var in target_var_lst if var != target_var]
    #df.drop(columns=cols_to_drop, inplace=True) # 종속변수
    #df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) 2진 분류화는 ipynb 파일에서 직접 수행

    # 종속변수를 데이터 프레임 맨 뒤로 옮기기
    cols = df.columns.tolist()
    cols = [col for col in cols if col != 'returns_next10m'] + ['returns_next10m'] # 종속변수 맨 뒤로
    df = df[cols]

    # 2. sequence 생성
    sequences = []
    scaler = MinMaxScaler()
    
    for start_idx in range(len(df) - seq_len + 1):  # 데이터 프레임을 순회하며 시퀀스 생성
        end_idx = start_idx + seq_len
        sequence = df.iloc[start_idx:end_idx]
        
        # 시퀀스 내에 del_idx가 1인 행이 있다면, 해당 시퀀스를 제외
        if sequence['del_idx'].sum() == 0:
            # 예측하고자 하는 마지막 피처의 값을 제외하고 스케일링
            scaled_sequence = scaler.fit_transform(sequence.drop(columns=['del_idx', target_var]))
            
            # 스케일링된 시퀀스에 예측하고자 하는 마지막 피처의 값을 추가
            scaled_sequence_with_target = pd.concat([pd.DataFrame(scaled_sequence), sequence[target_var].reset_index(drop=True)], axis=1)
            
            # 최종 시퀀스 추가
            sequences.append(scaled_sequence_with_target.values)
            
    # 3. X, y split
    sequences = np.array(sequences)
    # X와 y를 분리
    X = sequences[:, :, :-1] # 마지s막 시퀀스와 마지막 컬럼을 제외한 나머지
    y = sequences[:, -1, -1].reshape(-1, 1) # 각 시퀀스의 마지막 행, 마지막 컬럼의 값

    return X, y


In [5]:
# 데이터 불러오기
file_path = '../../data/' # 경로 설정
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
df = df.iloc[:10000]
df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) # 종속변수 이진분류화
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬
df = df.reset_index() # backtesting을 위한 index 값 생성

X, y= create_sequence(df, 20)

In [25]:
%%writefile sequence
def create_sequence_with_index(df, seq_len):
    sequences = []
    scaler = MinMaxScaler()
    sequence_indices = []  # 시퀀스의 마지막 행 인덱스를 저장할 리스트
    target_var = 'returns_next10m' # 종속변수

    
    for start_idx in range(len(df) - seq_len + 1):
        end_idx = start_idx + seq_len
        sequence = df.iloc[start_idx:end_idx]
        
        if sequence['del_idx'].sum() == 0:
            scaled_sequence = scaler.fit_transform(sequence.drop(columns=['del_idx', target_var]))
            scaled_sequence_with_target = pd.concat([pd.DataFrame(scaled_sequence), sequence[target_var].reset_index(drop=True)], axis=1)
            sequences.append(scaled_sequence_with_target.values)
            sequence_indices.append(sequence.index[-1])  # 마지막 행의 인덱스 추가
    
    sequences = np.array(sequences)
    X = sequences[:, :, :-1]
    y = sequences[:, -1, -1]
    indices = np.array(sequence_indices)  # 마지막 행 인덱스를 numpy 배열로 변환
    
    return X, y, indices


NameError: name 'model' is not defined

In [ ]:

# 함수 사용 예시
X, y, indices = create_sequence_with_index(df, seq_len=20)

# 모델 학습 및 예측 후
predicted = model.predict(X)

# 예측 결과가 1인 인덱스 찾기
predicted_indices = indices[predicted.flatten() == 1]

# 예측 결과가 1인 인덱스를 사용하여 원본 데이터에서 해당하는 return 값 추출
original_data_with_predictions = df.loc[predicted_indices]

# 백테스팅
# 예를 들어, 'returns_next10m'을 사용한 수익률 계산 등

In [30]:
print(X.shape)
print(y.shape)
print(indices.shape)

(4406, 20, 80)
(4406,)
(4406,)


In [21]:
# split (80% / 20%)
train_split = int(X.shape[0] * 0.8)
X[train_split:].shape

(882, 20, 80)

In [22]:
y[train_split:].shape

(882, 1)